# Librerias

In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings
import re
from unidecode import unidecode
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup

warnings.filterwarnings('ignore')

# Mis funciones

In [2]:
import import_ipynb
from mis_funciones import *

importing Jupyter notebook from mis_funciones.ipynb


# Extracción de urls y otros datos para la extracción masiva

In [4]:
urls_shows = []
locales_shows = []
nombre_shows = []
n_opiniones = []  # Lista para almacenar el número de opiniones

for i in range(1, 10):
    url = f'https://www.atrapalo.com/entradas/madrid/teatro-y-danza/monologos/p-{i}/'
    
    res1 = requests.get(url)
    sopa = BeautifulSoup(res1.content, 'html.parser')

    urls = sopa.find_all("a", {"class": "product-name"})
    for url_evento in urls:
        urls_shows.append(url_evento['href'])
        nombre_local = url_evento.find_next("a", {"class": "locality GATrackEvent_ubicacion hide-for-small-only"}).text
        locales_shows.append(nombre_local)

        nombre_show = url_evento.text.strip()
        nombre_shows.append(nombre_show)
        
        # Obtener el número de opiniones
        url_completa = 'https://www.atrapalo.com/' + url_evento['href']
        response = requests.get(url_completa)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Buscar el texto que contiene el número de opiniones
        texto_opiniones = soup.get_text()

        # Buscar un patrón de texto que incluya el número de opiniones
        match = re.search(r'(\d+)\s+opiniones', texto_opiniones)

        if match:
            # Extraer el número de opiniones
            num_opiniones = int(match.group(1))
            n_opiniones.append(num_opiniones)
        else:
            # Si no se encuentra el texto, agregar un valor predeterminado (por ejemplo, -1)
            n_opiniones.append(-1)



# Crear el DataFrame
df = pd.DataFrame({'url': urls_shows, 'local': locales_shows, 'nombre_show': nombre_shows, 'n_opiniones': n_opiniones})

df

,url,local,nombre_show,n_opiniones
0,/entradas/la-hora-y-media-de-el-club-de-la-com...,Teatro Príncipe Gran Vía,La Hora y Media de El Club de la Comedia Madrid,4140
1,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monólogos de El Golfo Comedy Club en Madrid,1731
2,/entradas/los-30-son-los-nuevos-vete-a-la-mier...,Teatro Bellas Artes,Los 30 son los nuevos ... ¡Vete a la mierda! ...,231
3,/entradas/tinder-sorpresa-andreu-casanova-en-m...,Cine Palacio de la Prensa - Sala 1,"Tinder Sorpresa - Andreu Casanova, en Madrid",1858
4,/entradas/tamagochi-un-monologo-millennial-de-...,Teatro Soho Club,Tamagochi: un monólogo Millennial de Silvia De...,81
...,...,...,...,...
316,/entradas/cena-y-espectaculo-soy-de-hacer-estu...,Hotel Campanile Madrid,Cena y Espectáculo: Soy de hacer Estudios con ...,-1
317,/entradas/open-mic-rision-imposible-_e4900715/,Savoy Leisure Atelier,Open Mic Risión Imposible,-1
318,/entradas/coolturetas-comedy-3-comicos-jam-mus...,Espacio Iroko,Coolturetas Comedy 3 Cómicos + Jam música en D...,107
319,/entradas/histericas_e4894914/,Estupenda Bar,Histéricas,20


Necesito unicamente los que aparecen en el siguiente data frame:

In [5]:
df_cuentas_locales = pd.read_csv("../01_data/cuentas_locales.csv", delimiter=",", header=0)
df_cuentas_locales

,nombre_show,tipo_show,precio,duracion,local,nota_atrapalo,numero_opiniones
0,Los Monólogos de El Golfo Comedy Club en Madrid,cuenta_local,5.0,95.0,El Golfo Comedy Club,8.9,1656.0
1,Noches de Comedia - Monólogos en Madrid,cuenta_local,6.0,75.0,El Sombrerero Comedy Club,9.5,901.0
2,Los Monólogos del Teatro de las Aguas,cuenta_local,5.0,60.0,Teatro de las Aguas,9.5,885.0
3,"Comedy Nights, monólogos en Beer Station",cuenta_local,8.0,80.0,Beer Station,9.0,778.0
4,Los Monólogos de Estupenda Café Bar,cuenta_local,2.4,80.0,Estupenda Bar,9.0,214.0
5,Monólogos de humor - Chocita del Loro Gran vía,cuenta_local,15.0,90.0,Teatro Chocita del Loro - Gran Vía,8.8,14386.0
6,Los Monólogos de Clandestino Café-Teatro Gran vía,cuenta_local,8.0,60.0,Clandestino Café Teatro,8.4,71.0
7,Estación Comedy,cuenta_local,2.5,60.0,Estación Malasaña,8.6,137.0
8,Monólogos en San Sebastián de los Reyes,cuenta_local,6.5,75.0,La Nota Rock,NaN,NaN


Y los shows que tengan df_show['local'] = df_cuentas_locales['local']

In [6]:
# Filtrar el DataFrame df para mantener solo las filas que coincidan con los locales en df_cuentas_locales['local']
df_filtrado = df[df['local'].isin(df_cuentas_locales['local'])]

# Añadir el nombre del espectáculo usando un bucle
df_filtrado['nombre_show'] = ''

for index, row in df_filtrado.iterrows():
    # Obtener la URL de la fila actual
    url = row['url']
    
    # Extraer el nombre del espectáculo de la URL
    nombre_show = url.split('/')[-2].replace('-', ' ').title()  # Obtener el nombre del espectáculo desde la URL
    
    # Limpiar el nombre del espectáculo (eliminar todo después de '_')
    nombre_show = nombre_show.split('_')[0] 
    
    # Asignar el nombre del espectáculo a la columna correspondiente
    df_filtrado.at[index, 'nombre_show'] = nombre_show

# Imprimir el DataFrame filtrado con el nombre del espectáculo
df_filtrado

,url,local,nombre_show,n_opiniones
1,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731
8,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263
12,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925
13,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425
17,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866
...,...,...,...,...
289,/entradas/malasana-comedy-show-_e4898595/,Estupenda Bar,Malasana Comedy Show,101
291,/entradas/histericas-en-san-sebastian-de-los-r...,La Nota Rock,Histericas En San Sebastian De Los Reyes,-1
292,/entradas/comedia-amateur_e4898596/,Estupenda Bar,Comedia Amateur,2
294,/entradas/pero-como-voy-a-ser-racista-si-mi-ga...,Estupenda Bar,Pero Como Voy A Ser Racista Si Mi Gato Es Negro,-1


In [10]:
# Calcular el número de páginas (son 10 comentarios por página)
df_filtrado['num_paginas'] = (df_filtrado['n_opiniones'] // 10) + 1
df_filtrado

,url,local,nombre_show,n_opiniones,num_paginas
1,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731,174
8,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263,127
12,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925,93
13,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425,43
17,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866,87
...,...,...,...,...,...
289,/entradas/malasana-comedy-show-_e4898595/,Estupenda Bar,Malasana Comedy Show,101,11
291,/entradas/histericas-en-san-sebastian-de-los-r...,La Nota Rock,Histericas En San Sebastian De Los Reyes,-1,0
292,/entradas/comedia-amateur_e4898596/,Estupenda Bar,Comedia Amateur,2,1
294,/entradas/pero-como-voy-a-ser-racista-si-mi-ga...,Estupenda Bar,Pero Como Voy A Ser Racista Si Mi Gato Es Negro,-1,0


In [11]:
# Cambio los'-1' por 0
df_filtrado['n_opiniones'] = df_filtrado['n_opiniones'].replace(-1, 0)
df_filtrado

,url,local,nombre_show,n_opiniones,num_paginas
1,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731,174
8,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263,127
12,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925,93
13,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425,43
17,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866,87
...,...,...,...,...,...
289,/entradas/malasana-comedy-show-_e4898595/,Estupenda Bar,Malasana Comedy Show,101,11
291,/entradas/histericas-en-san-sebastian-de-los-r...,La Nota Rock,Histericas En San Sebastian De Los Reyes,0,0
292,/entradas/comedia-amateur_e4898596/,Estupenda Bar,Comedia Amateur,2,1
294,/entradas/pero-como-voy-a-ser-racista-si-mi-ga...,Estupenda Bar,Pero Como Voy A Ser Racista Si Mi Gato Es Negro,0,0


In [23]:
df_filtrado.head(20)

,url,local,nombre_show,n_opiniones,num_paginas
1,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731,174
8,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263,127
12,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925,93
13,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425,43
17,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866,87
18,/entradas/los-monologos-del-teatro-de-las-agua...,Teatro de las Aguas,Los Monologos Del Teatro De Las Aguas,1387,139
19,/entradas/los-monologos-de-clandestino-comedia...,Clandestino Café Teatro,Los Monologos De Clandestino Comedia En Gran Via,95,10
20,/entradas/situaciones-incomodas-marcos-arizmen...,Clandestino Café Teatro,Situaciones Incomodas Marcos Arizmendi,13,2
23,/entradas/monologos-de-humor-chocita-del-loro-...,Teatro Chocita del Loro - Gran Vía,Monologos De Humor Chocita Del Loro Gran Via,14429,1443
24,/entradas/impro-monologos-clandestinos_e4887821/,Clandestino Café Teatro,Impro Monologos Clandestinos,32,4


# Guardamos el data frame de filtrado para el scraping masivo

In [24]:
# Especifico la ruta y el nombre del archivo CSV
ruta_archivo_csv = '../01_data/cuentas_scraping.csv'

# Guardo el DataFrame en un archivo CSV
df_filtrado.to_csv(ruta_archivo_csv, index=False)

# Extracción masiva con limpieza y guardado de datos

In [10]:
# Función para extraer comentarios de una página específica
def extraer_comentarios(url, nombre_show, nombre_local):
    """
    Esta función extrae comentarios de una página web y devuelve una lista de diccionarios con información detallada sobre cada comentario.
    
    Requiere las siguientes bibliotecas:
    - requests
    - BeautifulSoup (bs4)
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    comentarios = soup.select(".rws-results-review-item")
    lista_comentarios = []

    for comentario in comentarios:
        nota_total = comentario.select_one(".aui-ratingBadge--large").text
        fecha = comentario.select_one(".meta-info__date").text
        grupo_de = comentario.select_one(".meta-info__company .aui-badge-text").text

        aspectos = comentario.select(".rws-circle-chart-list__item")

        if aspectos:
            calidad = aspectos[0].select_one(".rws-circle-chart__value-container span").text
            puesta_en_escena = aspectos[1].select_one(".rws-circle-chart__value-container span").text
            interpretacion = aspectos[2].select_one(".rws-circle-chart__value-container span").text

            comentario_dict = {
                "fecha": fecha,
                "nota_total": nota_total,
                "grupo_de": grupo_de,
                "calidad_del_espectáculo": calidad,
                "puesta_en_escena": puesta_en_escena,
                "interpretación_artística": interpretacion,
                "nombre_show": nombre_show,  
                "nombre_local": nombre_local  
            }

            lista_comentarios.append(comentario_dict)

    return lista_comentarios

def convertir_fecha(fecha):
    meses = {
        "enero": "January",
        "febrero": "February",
        "marzo": "March",
        "abril": "April",
        "mayo": "May",
        "junio": "June",
        "julio": "July",
        "agosto": "August",
        "septiembre": "September",
        "octubre": "October",
        "noviembre": "November",
        "diciembre": "December"
    }
    partes = fecha.split()
    partes[1] = meses[unidecode(partes[1].lower())]
    fecha_nueva = " ".join(partes)
    return pd.to_datetime(fecha_nueva, format='%d %B %Y', errors='coerce')

## 1er día

In [22]:
# URL base de la página
base_url = "https://www.atrapalo.com"

# Inicializar lista para almacenar todos los comentarios
todos_los_comentarios = []

# Iterar a través de las filas de df_filtrado
for index, row in df_filtrado.iterrows():
    url = f"{base_url}{row['url']}"
    nombre_show = row['nombre_show']
    nombre_local = row['local']
    num_paginas = row['num_paginas']
    
    # Iterar a través de las páginas
    for i in range(1, num_paginas + 1):
        url_pagina = f"{url}page-{i}/"
        comentarios_pagina = extraer_comentarios(url_pagina, nombre_show, nombre_local)
        todos_los_comentarios.extend(comentarios_pagina)
    
    # Pasar a DataFrame los datos
    df_comentarios = pd.DataFrame(todos_los_comentarios)

    # Aplicar la función a la columna 'fecha'
    df_comentarios['fecha'] = df_comentarios['fecha'].apply(convertir_fecha)

    # Quitar 'sobre 10' de las columnas de calidad del espectáculo, puesta en escena e interpretación artística
    df_comentarios['calidad_del_espectáculo'] = df_comentarios['calidad_del_espectáculo'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['puesta_en_escena'] = df_comentarios['puesta_en_escena'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['interpretación_artística'] = df_comentarios['interpretación_artística'].str.replace(r'\s+sobre\s+10', '', regex=True)

    # Guardar el DataFrame en un archivo CSV
    df_comentarios.to_csv(f"../01_data/comentarios_{nombre_show}.csv", index=False)
    
# Se ha quedado en la chocita

IndexError: list index out of range

## 2º día

En la anterior linea de código se quedo en 'Monologos De Humor Chocita Del Loro Gran Via'.

In [5]:
# Volvemos a cargar el archivo de filtrado
df_filtrado = pd.read_csv("../01_data/cuentas_scraping.csv", delimiter=",", header=0)
df_filtrado

,url,local,nombre_show,n_opiniones,num_paginas
0,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731,174
1,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263,127
2,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925,93
3,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425,43
4,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866,87
...,...,...,...,...,...
129,/entradas/malasana-comedy-show-_e4898595/,Estupenda Bar,Malasana Comedy Show,101,11
130,/entradas/histericas-en-san-sebastian-de-los-r...,La Nota Rock,Histericas En San Sebastian De Los Reyes,0,0
131,/entradas/comedia-amateur_e4898596/,Estupenda Bar,Comedia Amateur,2,1
132,/entradas/pero-como-voy-a-ser-racista-si-mi-ga...,Estupenda Bar,Pero Como Voy A Ser Racista Si Mi Gato Es Negro,0,0


Hemos observado que hay cierto número de opiniones que se han scrapeado mal y tienen un '10' extra delante. Lo limpiamos:

In [6]:
# Iterar sobre las filas del DataFrame
for index, row in df_filtrado.iterrows():
    # Convertir n_opiniones a string para verificar si comienza con '10'
    n_opiniones = str(row['n_opiniones'])
    
    # Verificar si comienza con '10'
    if n_opiniones.startswith('10'):
        # Quitar '10' del inicio
        df_filtrado.at[index, 'n_opiniones'] = int(n_opiniones[2:])

# Ver el DataFrame actualizado
df_filtrado

,url,local,nombre_show,n_opiniones,num_paginas
0,/entradas/los-monologos-de-el-golfo-comedy-clu...,El Golfo Comedy Club,Los Monologos De El Golfo Comedy Club En Madrid,1731,174
1,/entradas/malasana-los-mejores-monologos-de-ma...,Estación Malasaña,Malasana Los Mejores Monologos De Madrid,1263,127
2,/entradas/noches-de-comedia-monologos-en-madri...,El Sombrerero Comedy Club,Noches De Comedia Monologos En Madrid,925,93
3,/entradas/impro-fever_e4874545/,Teatro de las Aguas,Impro Fever,425,43
4,/entradas/aguantame-el-cubata-que-me-da-la-ris...,Teatro de las Aguas,Aguantame El Cubata Que Me Da La Risa Juan Aroca,866,87
...,...,...,...,...,...
129,/entradas/malasana-comedy-show-_e4898595/,Estupenda Bar,Malasana Comedy Show,1,11
130,/entradas/histericas-en-san-sebastian-de-los-r...,La Nota Rock,Histericas En San Sebastian De Los Reyes,0,0
131,/entradas/comedia-amateur_e4898596/,Estupenda Bar,Comedia Amateur,2,1
132,/entradas/pero-como-voy-a-ser-racista-si-mi-ga...,Estupenda Bar,Pero Como Voy A Ser Racista Si Mi Gato Es Negro,0,0


Como 'La chocita del loro' dío problemas seguimos por el siguiente regestro. Para ello buscamos su indice:

In [15]:
indice_reanudacion = df_filtrado[df_filtrado['nombre_show'] == 'Impro Monologos Clandestinos'].index[0]
indice_reanudacion

9

Reanudamos el scraping:

In [16]:
# URL base de la página
base_url = "https://www.atrapalo.com"

# Inicializar lista para almacenar todos los comentarios
todos_los_comentarios = []

# Iterar a través de las filas de df_filtrado desde el índice de reanudación
for index, row in df_filtrado.iloc[indice_reanudacion:].iterrows():
    url = f"{base_url}{row['url']}"
    nombre_show = row['nombre_show']
    nombre_local = row['local']
    num_paginas = row['num_paginas']
    
    # Iterar a través de las páginas
    for i in range(1, num_paginas + 1):
        url_pagina = f"{url}page-{i}/"
        comentarios_pagina = extraer_comentarios(url_pagina, nombre_show, nombre_local)
        todos_los_comentarios.extend(comentarios_pagina)
    
    # Pasar a DataFrame los datos
    df_comentarios = pd.DataFrame(todos_los_comentarios)

    # Aplicar la función a la columna 'fecha'
    df_comentarios['fecha'] = df_comentarios['fecha'].apply(convertir_fecha)

    # Quitar 'sobre 10' de las columnas de calidad del espectáculo, puesta en escena e interpretación artística
    df_comentarios['calidad_del_espectáculo'] = df_comentarios['calidad_del_espectáculo'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['puesta_en_escena'] = df_comentarios['puesta_en_escena'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['interpretación_artística'] = df_comentarios['interpretación_artística'].str.replace(r'\s+sobre\s+10', '', regex=True)

    # Guardar el DataFrame en un archivo CSV
    df_comentarios.to_csv(f"../01_data/comentarios_{nombre_show}.csv", index=False)

# Unimos los csv y creamos una sola BBDD

In [24]:
# Volvemos a cargar el archivo de filtrado
df1 = pd.read_csv("../01_data/comentarios_Situaciones Incomodas Marcos Arizmendi.csv", delimiter=",", header=0)
df2 = pd.read_csv("../01_data/comentarios_Histericas.csv", delimiter=",", header=0)
df_comentarios = pd.concat([df1, df2], ignore_index=True)
df_comentarios

,fecha,nota_total,grupo_de,calidad_del_espectáculo,puesta_en_escena,interpretación_artística,nombre_show,nombre_local
0,2023-10-10,10.0,Pareja,10.0,10.0,10.0,Los Monologos De El Golfo Comedy Club En Madrid,El Golfo Comedy Club
1,2023-08-17,10.0,Amigos,10.0,10.0,10.0,Los Monologos De El Golfo Comedy Club En Madrid,El Golfo Comedy Club
2,2023-10-13,10.0,Pareja,10.0,10.0,10.0,Los Monologos De El Golfo Comedy Club En Madrid,El Golfo Comedy Club
3,2023-08-17,9.1,Pareja,10.0,10.0,7.5,Los Monologos De El Golfo Comedy Club En Madrid,El Golfo Comedy Club
4,2023-10-09,10.0,Amigos,10.0,10.0,10.0,Los Monologos De El Golfo Comedy Club En Madrid,El Golfo Comedy Club
...,...,...,...,...,...,...,...,...
174601,2023-02-02,10.0,Solo,10.0,10.0,10.0,Histericas,Estupenda Bar
174602,2023-01-30,7.5,Amigos,7.5,7.5,7.5,Histericas,Estupenda Bar
174603,2022-11-22,10.0,Amigos,10.0,10.0,10.0,Histericas,Estupenda Bar
174604,2023-10-02,5.0,Amigos,5.0,5.0,5.0,Histericas,Estupenda Bar


In [25]:
# Guardo el DataFrame en un archivo CSV
df_comentarios.to_csv('../01_data/comentarios_atrapalo.csv', index=False)